In [1]:
import japanize_matplotlib

# plot
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import yfinance as yf
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from stockstats import StockDataFrame

sns.set(font="IPAexGothic", rc={"figure.figsize": (11, 8)})
pd.options.display.float_format = "{:6.2f}".format

/usr/local/lib/python3.9/site-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
# Valid start and end: YYYY-MM-DD
# Valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
# Valid intervals: [1m, 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1wk, 1mo, 3mo]
response = yf.download(
    tickers="SOXL",
    period="5y",
    interval="1d",
    group_by="ticker",
)

[*********************100%***********************]  1 of 1 completed


In [12]:
yfdata = response.copy().dropna()
# yfdata = yfdata["1950-01":"202１-12"]  # 直近の暴落を除いて検証する
yfdata

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2017-05-22,6.01,6.12,5.99,6.11,6.00,5236500
2017-05-23,6.13,6.13,5.89,6.04,5.93,4482000
2017-05-24,6.14,6.19,6.07,6.15,6.05,3240000
2017-05-25,6.24,6.32,6.13,6.26,6.15,3831000
2017-05-26,6.26,6.34,6.12,6.33,6.22,4323000
...,...,...,...,...,...,...
2022-05-13,20.57,22.91,20.18,22.49,22.49,65648400
2022-05-16,21.86,22.51,21.20,21.37,21.37,49300800
2022-05-17,23.08,24.74,22.84,24.58,24.58,65619700


In [13]:
def convert_df_to_stock_df(df: pd.DataFrame) -> StockDataFrame:
    sdf = df.copy()
    sdf.rename(
        columns={
            "Open": "open",
            "High": "high",
            "Low": "low",
            "Close": "close",
            "Adj Close": "amount",
            "Volume": "volume",
        },
        inplace=True,
    )
    sdf.index.names = ["date"]
    return StockDataFrame(sdf)


def MACD(
    arr: pd.DataFrame, ema_short_period: int, ema_long_period: int, signal: int
) -> tuple[pd.Series, pd.Series]:
    sdf = convert_df_to_stock_df(arr)
    StockDataFrame.MACD_EMA_SHORT = 12
    StockDataFrame.MACD_EMA_LONG = 26
    StockDataFrame.MACD_EMA_SIGNAL = 9
    return (sdf["macd"], sdf["macds"])


def RSI(arr: pd.DataFrame, rsi: int) -> pd.Series:
    sdf = convert_df_to_stock_df(arr)
    return sdf["rsi_" + str(rsi)]

In [14]:
class My_Strategy(Strategy):
    # MACD
    use_macd_buy = 1
    use_macd_sell = 1
    prop_short = 12
    prop_long = 26
    prop_signal = 9

    # RSI
    use_rsi_buy = 1
    use_rsi_sell = 1
    prop_rsi = 14
    prop_sell_with_rsi = 70
    prop_buy_with_rsi = 30

    def init(self):
        self.macd, self.macd_signal = self.I(
            MACD, self.data.df, self.prop_short, self.prop_long, self.prop_signal
        )

        self.rsi = self.I(RSI, self.data.df, self.prop_rsi)

    def next(self):
        
        macd_week = 
        macds_weel

        # 計算できていない場合トレードしない
        if len(self.data.index) < self.prop_long:
            return

        if crossover(self.macd, self.macd_signal) and self.use_macd_buy == 1:
            self.buy()
            return

        if self.prop_buy_with_rsi > self.rsi[-1] and self.use_rsi_buy == 1:
            self.buy()
            return

        if crossover(self.macd_signal, self.macd) and self.use_macd_sell == 1:
            self.position.close()
            return

        if self.prop_sell_with_rsi < self.rsi[-1] and self.use_rsi_sell == 1:
            self.position.close()
            return


bt = Backtest(yfdata, My_Strategy, cash=10000, commission=0.002, exclusive_orders=True)

# 最適化
optimize = bt.optimize(
    # prop_sell_with_rsi=range(10, 90, 5),
    # prop_buy_with_rsi=range(10, 90, 5),
    # constraint=lambda p: p.prop_buy_with_rsi < p.prop_sell_with_rsi,
    prop_rsi=[7, 14, 21],
    use_macd_buy=[0, 1],
    use_macd_sell=[0, 1],
    use_rsi_buy=[0, 1],
    use_rsi_sell=[0, 1],
    method="grid",  # unuse model-based optimization
    maximize="Equity Final [$]",
)

bt.plot()
print(optimize)
print(optimize._strategy)

# 出力
# output = bt.run()
# print(output)

# print(output["_trades"])
# bt.plot()

Start                     2017-05-22 00:00:00
End                       2022-05-19 00:00:00
Duration                   1823 days 00:00:00
Exposure Time [%]                       49.09
Equity Final [$]                     47603.01
Equity Peak [$]                     174978.88
Return [%]                             376.03
Buy & Hold Return [%]                  235.57
Return (Ann.) [%]                       36.66
Volatility (Ann.) [%]                   88.78
Sharpe Ratio                             0.41
Sortino Ratio                            0.91
Calmar Ratio                             0.50
Max. Drawdown [%]                      -72.79
Avg. Drawdown [%]                       -9.88
Max. Drawdown Duration      336 days 00:00:00
Avg. Drawdown Duration       28 days 00:00:00
# Trades                                   45
Win Rate [%]                            57.78
Best Trade [%]                          60.35
Worst Trade [%]                        -28.09
Avg. Trade [%]                    